In [71]:
import torch
import argparse
from tqdm import tqdm
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import yaml
from argparse import ArgumentParser
import os

# from datasets.quora_bert_mask_predict_dataset import QuoraBertMaskPredictDataset
# from datasets.quora_bert_dataset import QuoraBertDataset
from transformer.Models import Transformer
from transformer.Translator import Translator
from utils import same_seeds
from utils import cal_loss, cal_performance, log_performances

In [29]:
# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
# parse argument
parser = ArgumentParser()
parser.add_argument("--config_path", dest="config_path",
                    default='../configs/dpng_transformer_bert_tokenizer_bow_indivtopk.yaml')
parser.add_argument("--preprocessed", dest="preprocessed", action="store_true")
parser.add_argument("--seed", dest="seed", default=0, type=int)

args = parser.parse_args()
config_path = args.config_path
preprocessed = args.preprocessed
seed = args.seed
print("config_path:", config_path)
print("preprocessed: ", preprocessed)
print("seed: ", seed)

usage: ipykernel_launcher.py [-h] [--config_path CONFIG_PATH] [--preprocessed]
                             [--seed SEED]
ipykernel_launcher.py: error: unrecognized arguments: -f /shared_home/r08922168/.local/share/jupyter/runtime/kernel-d90c4d4a-8568-449c-b73d-37d4c9bd157a.json


SystemExit: 2

/shared_home/r08922168/miniconda3/envs/my_paraphrase_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3425: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [60]:
# fix seed
# seed = 0
same_seeds(seed)

In [61]:
##### Read Arguments from Config File #####

# config_path = '../configs/dpng_transformer_bert_tokenizer.yaml'
# config_path = '../configs/dpng_transformer_bert_tokenizer_bow.yaml'
# config_path = '../configs/dpng_transformer_bert_tokenizer_bow_indivtopk.yaml'
# config_path = '../configs/dpng_transformer_bert_tokenizer_bow_indivtopk_onlybow.yaml'
# config_path = '../configs/dpng_transformer_bert_tokenizer_bow_indivtopk_replace.yaml'
# config_path = '../configs/dpng_transformer_bert_tokenizer_bow_indivtopk_replace_nopreprocess.yaml'


with open(config_path) as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    print(config)

save_model_path = config['save_model_path']
output_file = config['test_output_file']
use_dataset = config['dataset']

batch_size = 1

d_model = config['d_model']
d_inner_hid = config['d_inner_hid']
d_k = config['d_k']
d_v = config['d_v']

n_head = config['n_head']
n_layers = config['n_layers']
n_warmup_steps = config['n_warmup_steps']

dropout = config['dropout']
embs_share_weight = config['embs_share_weight']
proj_share_weight = config['proj_share_weight']
label_smoothing = config['label_smoothing']

train_size = config['train_size']
val_size = config['val_size']
test_size = config['test_size']

beam_size = 3
max_seq_len = 30

try:
    is_bow = config['is_bow']

    if is_bow:
        bow_strategy = config['bow_strategy']
        topk = config['topk']
        if bow_strategy != 'simple_sum':
            indiv_topk = config['indiv_topk']
        else:
            # not used but use default value for simplicity
            indiv_topk = 50
            
        only_bow = config['only_bow']
        replace_predict = config['replace_predict']
        append_bow = config['append_bow']

except KeyError:
    is_bow = False
    
try:
    only_bow = config['only_bow']
except KeyError:
    only_bow = False 
# ###################

{'save_model_path': '../models/DNPG_base_transformer_bert_tokenizer_bert_bow_indivtopk_replace_nopreprocess.pth', 'log_file': '../logs/DNPG_base_transformer_bert_tokenizer_training_bert_bow_indiv_topk_replace_nopreprocess.txt', 'test_output_file': '../outputs/test_DNPG_transformer_bert_tokenizer_bow_indivtopk_replace_out_nopreprocess.txt', 'val_output_file': '../outputs/val_DNPG_transformer_bert_tokenizer_bow_indivtopk_replace_out_nopreprocess.txt', 'dataset': 'quora_bert_mask_predict_dataset', 'num_epochs': 50, 'batch_size': 100, 'd_model': 450, 'd_inner_hid': 512, 'd_k': 50, 'd_v': 50, 'n_head': 9, 'n_layers': 3, 'n_warmup_steps': 12000, 'dropout': 0.1, 'embs_share_weight': True, 'proj_share_weight': True, 'label_smoothing': False, 'train_size': 100000, 'val_size': 4000, 'test_size': 20000, 'is_bow': True, 'bow_strategy': 'indiv_topk', 'indiv_topk': 10, 'topk': 25, 'lr': '1e-3', 'only_bow': False, 'replace_predict': True, 'append_bow': True}


In [62]:
# set model and log path
seed_model_root = '../models/fixseed/seed{}/'.format(seed)
seed_output_root = '../outputs/fixseed/seed{}/'.format(seed)

if not os.path.exists(seed_model_root):
    os.makedirs(seed_model_root)

if not os.path.exists(seed_output_root):
    os.makedirs(seed_output_root)

save_model_path = seed_model_root + save_model_path.split('/')[-1]
output_file = seed_output_root + output_file.split('/')[-1]
print('seed: ', seed)
print('save model path: ', save_model_path)
print('output file: ', output_file)

seed:  0
save model path:  ../models/fixseed/seed0/DNPG_base_transformer_bert_tokenizer_bert_bow_indivtopk_replace_nopreprocess.pth
output file:  ../outputs/fixseed/seed0/test_DNPG_transformer_bert_tokenizer_bow_indivtopk_replace_out_nopreprocess.txt


In [63]:
# load dataset
preprocessed = False

from datasets.quora_text_dataset import QuoraTextDataset

if preprocessed:
    from datasets.quora_preprocessed_dataset import QuoraPreprocessedDataset as Dataset
else:
    if use_dataset == 'quora_dataset':
        from datasets.quora_dataset import QuoraDataset as Dataset
    elif use_dataset == 'quora_bert_dataset':
        from datasets.quora_bert_dataset import QuoraBertDataset as Dataset
    elif use_dataset == 'quora_bert_mask_predict_dataset':
        from datasets.quora_bert_mask_predict_dataset import QuoraBertMaskPredictDataset as Dataset
    elif use_dataset == 'quora_word_mask_prediction_dataset':
        from datasets.quora_word_mask_prediction_dataset import QuoraWordMaskPredictDataset as Dataset
    else:
        raise NotImplementedError("Dataset is not defined or not implemented")
        


In [79]:
def create_mini_batch(samples):
    seq1_tensors = [s[0] for s in samples]
    seq2_tensors = [s[1] for s in samples]

    # zero pad
    seq1_tensors = pad_sequence(seq1_tensors,
                                  batch_first=True)

    seq2_tensors = pad_sequence(seq2_tensors,
                                  batch_first=True)    
    
    return seq1_tensors, seq2_tensors

# force not to replace the tokens when predicting
# replace_predict = False

if preprocessed:
    model_name = config_path.split('/')[-1][:-5]
    preprocessed_file = '../data/preprocess_all_{}.npy'.format(model_name)
    dataset = Dataset("test", train_size, val_size, test_size, preprocessed_file=preprocessed_file)
elif is_bow:
    dataset = Dataset(
        "test", train_size, val_size, test_size, bow_strategy=bow_strategy, topk=topk, 
        indiv_topk=indiv_topk, only_bow=only_bow, use_origin=only_bow, 
        replace_predict=False, append_bow=append_bow
    )
#     dataset = Dataset(
#         "val", train_size, val_size, test_size, bow_strategy=bow_strategy, topk=topk, 
#         indiv_topk=indiv_topk, only_bow=only_bow, use_origin=only_bow, 
#         replace_predict=False, append_bow=append_bow
#     )
else:
    dataset = Dataset("test", train_size, val_size, test_size)

same_seeds(seed)
text_dataset = QuoraTextDataset("test", train_size, val_size, test_size)
data_loader = DataLoader(dataset, batch_size=batch_size, collate_fn=create_mini_batch, shuffle=False)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [66]:
# # text_dataset[0]
# i = 0
# idxs, idxs2 = dataset[i]
# print(text_dataset[i])
# print(dataset.tokenizer.convert_ids_to_tokens(idxs))
# print(dataset.tokenizer.convert_ids_to_tokens(idxs2))


('What are the benefits of using digital signage for your business ?', 'What would digital signage do for your business ? ')
['[CLS]', 'What', 'are', 'the', 'benefits', 'of', 'using', 'digital', 'sign', '##age', 'for', 'your', 'business', '?', '[SEP]', 'the', 'in', 'your', 'of', 'for', 'to', '.', 'on', 'about', 'a', '?', 'our', 'this', 'broker', 'are', ',', '##age', 'What', 'sign', 'advantages', 'link', 'were', 'benefits', 'chances', 'mile', '[SEP]']
['[CLS]', 'What', 'would', 'digital', 'sign', '##age', 'do', 'for', 'your', 'business', '?', '[SEP]']


In [67]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transformer = Transformer(
    dataset.n_words,
    dataset.n_words,
    src_pad_idx=dataset.PAD_token_id,
    trg_pad_idx=dataset.PAD_token_id,
    trg_emb_prj_weight_sharing=proj_share_weight,
    emb_src_trg_weight_sharing=embs_share_weight,
    d_k=d_k,
    d_v=d_v,
    d_model=d_model,
    d_word_vec=d_model,
    d_inner=d_inner_hid,
    n_layers=n_layers,
    n_head=n_head,
    dropout=dropout    
)

model = transformer.to(device)

model.load_state_dict((torch.load(
        save_model_path, map_location=device)))

<All keys matched successfully>

In [ ]:
# eval, replace predict
# 1.0310528681413182, 0.7787169811320754

# eval, no replace predict
# 2.117869621312843, 0.5975849056603774
# test
# 2.???, 0.59???

In [68]:
src_pad_idx = dataset.PAD_token_id
trg_pad_idx = dataset.PAD_token_id
    
trg_bos_idx = dataset.SOS_token_id
trg_eos_idx = dataset.EOS_token_id
unk_idx = dataset.UNK_token_id

In [69]:
translator = Translator(
        model=model,
        beam_size=beam_size,
        max_seq_len=max_seq_len,
        src_pad_idx=src_pad_idx,
        trg_pad_idx=trg_pad_idx,
        trg_bos_idx=trg_bos_idx,
        trg_eos_idx=trg_eos_idx).to(device)

In [70]:
tokenizer = dataset.tokenizer
# output_file = '../outputs/tmp.txt'
with open(output_file, 'w') as f:
    for i, (seq1, seq2) in enumerate(tqdm(data_loader)):
        src_line, trg_line = text_dataset.sentences[i]
        
        input_seq = seq1.to(device)
        trg_seq = seq2[0].tolist()
        
        pred_seq = translator.translate_sentence(input_seq)
        pred_tokens = tokenizer.convert_ids_to_tokens(pred_seq)
        pred_line = tokenizer.convert_tokens_to_string(pred_tokens)
        pred_line = pred_line.replace(dataset.SOS_token, '').replace(dataset.EOS_token, '')
            
        input_tokens = tokenizer.convert_ids_to_tokens(input_seq[0].tolist())
        input_line = tokenizer.convert_tokens_to_string(input_tokens)
#         input_line = input_line.replace(dataset.SOS_token, '').replace(dataset.EOS_token, '')
        
        
        trg_tokens = tokenizer.convert_ids_to_tokens(trg_seq)
        trg_line = tokenizer.convert_tokens_to_string(trg_tokens)
        trg_line = trg_line.replace(dataset.SOS_token, '').replace(dataset.EOS_token, '')

#         print('*' * 80)
#         print('Source: ', src_line.strip())
#         print('Target: ', trg_line.strip())
#         print('Input: ', input_line.strip())
#         print('Predict: ', pred_line.strip())
        
        f.write('*' * 80)
        f.write('\n')
        f.write('Source: {}\n'.format(src_line.strip()))
        f.write('Target: {}\n'.format(trg_line.strip()))
        f.write('Input: {}\n'.format(input_line.strip()))
        f.write('Predict: {}\n'.format(pred_line.strip()))
        f.flush()



  0%|          | 2/20000 [00:00<25:07, 13.27it/s]

********************************************************************************
Source:  What are the benefits of using digital signage for your business ?
Target:  What would digital signage do for your business ?
Input:  [CLS] What are the benefits of using digital signage for your business ? [SEP] the in your of for to . on about a ? our this broker are ,age What sign advantages link were benefits chances mile [SEP]
Predict:  What are the benefits of a mileage ?
********************************************************************************
Source:  How can I speak faster ?
Target:  How can you speak faster ?
Input:  [CLS] How can I speak faster ? [SEP] . you ? it How that could can go do move run walk be did ! I get he she But would travel think Why [SEP]
Predict:  Is time travel possible ? If yes , how ?
********************************************************************************
Source:  How do I tell a girl I like her ?
Target:  What is the best way to tell a girl that you

  0%|          | 6/20000 [00:00<23:41, 14.07it/s]

********************************************************************************
Source:  How can you lose weight quickly ?
Target:  How do I lose weight and reduce my waist quickly ?
Input:  [CLS] How can you lose weight quickly ? [SEP] ? lose too How that do can gain did could But so . I shed ! will shift you how would she And lost he [SEP]
Predict:  How can I remember my lost person ?
********************************************************************************
Source:  How can I travel the world for free ?
Target:  What is the best way to travel around the world for free ?
Input:  [CLS] How can I travel the world for free ? [SEP] that this . you ? her , the could can How world do did ! for I would rule see leave But run will we [SEP]
Predict:  How can we save the world ?
********************************************************************************
Source:  What are some good hosts that have free website builders ?
Target:  What is the best free mobile website builder ?
Input:

  0%|          | 8/20000 [00:00<40:56,  8.14it/s]

...

  0%|          | 66/20000 [00:06<27:26, 12.11it/s]

********************************************************************************
Source:  What are the seven deadly sins ?
Target:  What are the seven deadly sins ? Doctor Faustus ?
Input:  [CLS] What are the seven deadly sins ? [SEP] ? , What the . Where about ! of deadly these sins were those are most ; weapons truly your These other Who Those what [SEP]
Predict:  What is the most weapons ?
********************************************************************************
Source:  Why do I find medieval history so interesting ?
Target:  Why is medieval history so interesting ?
Input:  [CLS] Why do I find medieval history so interesting ? [SEP] this find ? Why do did so you I How would should make fascinating . found does keep why miss think ! confusing people interesting [SEP]
Predict:  Why do people find everything interesting ?
********************************************************************************
Source:  What is the easiest way to crack gate ?
Target:  How I do crack gate

  0%|          | 69/20000 [00:06<30:44, 10.80it/s]

********************************************************************************
Source:  What are some of the best songs by Linkin Park ?
Target:  What is the best Linkin Park song ?
Input:  [CLS] What are some of the best songs by Linkin Park ? [SEP] of Park about in , Linkin more ? the some with and say . on songs are from byins were ! Where many [SEP]
Predict:  What are some songs on Linkin ?
********************************************************************************
Source:  Which is currently the best IAS coaching centre in Hyderabad ?
Target:  Could anyone help me with the best IAS coaching centers in Hyderabad ?
Input:  [CLS] Which is currently the best IAS coaching centre in Hyderabad ? [SEP] in ,M ofC . at IndiaCT is the I U ; What IF training AIB IT O BC !R only [SEP]
Predict:  What is the best MPR training course at BCA ?


KeyboardInterrupt: 